In [0]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers, models, layers, regularizers, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

print('\nLoading Data from file. ', end='')
lab = np.load('/content/drive/My Drive/Colab Notebooks/cs_labels.npz')
imgs = np.load('/content/drive/My Drive/Colab Notebooks/AP_dataset_float1RGB.npz')
print('\n\tTraining Data...', end='')
x_train = imgs['x_test']
y_train = lab['y_cs_train']
#y_train = data['y_train']
print('Complete.\n\tValidation Data...', end='')
x_val = imgs['x_val']
y_val = lab['y_cs_val']
# y_val = data['y_val']
print('Complete.\n\tTest Data...', end='')
x_test = imgs['x_train']
y_test = lab['y_cs_test']
# y_test = data['y_test']
print('Complete.')

# model = load_model('/content/drive/My Drive/Colab Notebooks/GateNet6.h5')

Found GPU at: /device:GPU:0

Loading Data from file. 
	Training Data...Complete.
	Validation Data...Complete.
	Test Data...Complete.


In [0]:
def ti_84(tl, vl, test_data): # under construction
  epochs = range(1, len(tl) + 1)
  plt.figure()
  plt.title('Training and Validation loss per Epoch with Test Score')
  plt.plot(epochs, tl, c='b', ls='-', label='Training MSE')
  plt.plot(epochs, vl, c='g', ls='--', label='Validation MSE')
  plt.axhline(test_data[1], c='r', label='Test MAE Score')
  plt.legend()
  plt.show()

In [0]:
def vis(model, num_pic):
  # model = load_model(model_path)
  for i in range(num_pic):
    prediction = model.predict(x_test[np.newaxis, i])
    x_pred = prediction[0, 0::2]
    y_pred = prediction[0, 1::2]
    x_true = y_test[i, 0::2]
    y_true = y_test[i, 1::2]
    plt.figure()
    plt.imshow(x_test[i])
    plt.scatter(x_pred, y_pred, s=60, c='r')
    plt.scatter(x_true, y_true, s=60, c='g')
    plt.show()

In [0]:
def naive_model():
  net = models.Sequential()
  # BLOCK 1
  net.add(layers.SeparableConv2D(128, (7, 7), depthwise_regularizer=regularizers.l2(1e-4), activation='relu', input_shape=(216, 324, 3) ))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(128, (7, 7), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(128, (7, 7), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.MaxPooling2D((4, 4)))
  # BLOCK 2
  net.add(layers.SeparableConv2D(64, (5, 5), depthwise_regularizer=regularizers.l2(1e-4), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(64, (5, 5), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(64, (5, 5), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.GlobalMaxPooling2D())
  # REGRESSOR
  net.add(layers.Dense(64, activation='relu'))
  net.add(layers.Dense(64, activation='relu'))
  net.add(layers.Dense(8))
  # net.summary()
  return net

  # log error was bad for spacial awareness and depth
  # MSE was good for spacial awareness not good for depth

In [0]:
def educated_model():
  net = models.Sequential()
  # BLOCK 1
  net.add(layers.SeparableConv2D(128, (7, 7), activation='relu', input_shape=(216, 324, 3)))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(128, (7, 7), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(128, (7, 7), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.MaxPooling2D((2, 2)))
  # BLOCK 2
  net.add(layers.SeparableConv2D(64, (5, 5), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(64, (5, 5), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(64, (5, 5), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.MaxPooling2D((2, 2)))
  # BLOCK 3
  net.add(layers.SeparableConv2D(32, (3, 3), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(32, (3, 3), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.MaxPooling2D((4, 4)))
  # BLOCK 4
  net.add(layers.SeparableConv2D(32, (3, 3), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.SeparableConv2D(32, (3, 3), activation='relu'))
  net.add(layers.BatchNormalization())
  net.add(layers.GlobalMaxPooling2D())
  # REGRESSOR
  net.add(layers.Dense(64, activation='relu'))
  net.add(layers.Dense(64, activation='relu'))
  net.add(layers.Dense(8))
  return net

In [0]:
def functional_model():
  gate_input = Input(shape=(216, 324, 3), dtype='float32', name='Gates')

  branch_1 = layers.Conv2D(128, 5, activation='relu', strides=2)(gate_input)
  branch_1 = layers.MaxPooling2D((2, 2))(branch_1)

  branch_2 = layers.Conv2D(128, 3, activation='relu')(gate_input)
  branch_2 = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_2)
  branch_2 = layers.MaxPooling2D((2, 2))(branch_2)

  branch_3 = layers.AveragePooling2D(2, strides=2)(gate_input)
  branch_3 = layers.Conv2D(128, 3, activation='relu')(branch_3)
  branch_3 = layers.MaxPooling2D((2, 2))(branch_3)

  branch_4 = layers.Conv2D(128, 1, activation='relu')(gate_input)
  branch_4 = layers.Conv2D(128, 3, activation='relu')(branch_4)
  branch_4 = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_4)
  branch_4 = layers.MaxPooling2D((2, 2))(branch_4)

  concatenate = layers.concatenate([branch_1, branch_2, branch_3, branch_4], axis=-1)

  regressor = layers.Flatten()(concatenate)
  dense = layers.Dense(64, activation='relu')(regressor)
  gate_output = layers.Dense(8, name='Gate_Points')(dense)

  net = models.Model(gate_input, gate_output)
  return net

In [0]:
def residual_model():
  gate_input = Input(shape=(216, 324, 3), dtype='float32', name='Gates')
  Conv1 = layers.Conv2D(128, 3, activation='relu', padding='same')(gate_input)
  Norm1 = layers.BatchNormalization()(Conv1)
  Conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(Norm1)
  Norm2 = layers.BatchNormalization()(Conv2)
  Conv3 = layers.Conv2D(128, 3, activation='relu', padding='same')(Norm2)
  Norm3 = layers.BatchNormalization()(Conv3)
  Conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(Norm3)
  Pool1 = layers.MaxPooling2D(2)(Conv4)
  ResDS = layers.Conv2D(128, 1, strides=2, padding='same')(Conv1)
  ResLayer = layers.add([Pool1, ResDS])
  Conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(ResLayer)
  Pool2 = layers.MaxPooling2D(4)(Conv5)
  flatten = layers.Flatten()(Pool2)
  Dense1 = layers.Dense(512, activation='relu')(flatten)
  Dense2 = layers.Dense(216, activation='relu')(Dense1)
  gate_output = layers.Dense(8)(Dense2)

  net = models.Model(gate_input, gate_output)
  return net

In [0]:
def better_residual_model():
  gate_input = Input(shape=(216, 324, 3), dtype='float32', name='Gates')

  Branch1 = layers.Conv2D(64, 1, activation='relu', strides=4, padding='same')(gate_input)

  Branch2 = layers.Conv2D(64, 5, activation='relu', padding='same')(gate_input)
  Branch2 = layers.Conv2D(64, 5, activation='relu', padding='same')(Branch2)
  Branch2 = layers.Conv2D(32, 1, activation='relu', strides=2, padding='same')(Branch2)
  Branch2 = layers.MaxPooling2D(2)(Branch2)

  Branch3 = layers.AveragePooling2D(2, strides=2)(gate_input)
  Branch3 = layers.Conv2D(64, 3, activation='relu', padding='same')(Branch3)
  Branch3 = layers.MaxPooling2D(2)(Branch3)

  Branch4 = layers.Conv2D(64, 1, activation='relu', padding='same', strides=2)(gate_input)
  Branch4 = layers.Conv2D(64, 3, activation='relu', padding='same')(Branch4)
  Branch4 = layers.Conv2D(32, 3, activation='relu', padding='same')(Branch4)
  Branch4 = layers.MaxPooling2D(2)(Branch4)

  Branch5 = layers.MaxPooling2D(2, strides=2)(gate_input)
  Branch5 = layers.Conv2D(64, 3, activation='relu', padding='same')(Branch5)
  Branch5 = layers.AveragePooling2D(2)(Branch5)

  join = layers.concatenate([Branch1, Branch2, Branch3, Branch4, Branch5], axis=-1)

  regressor = layers.Flatten()(join)
  dense = layers.Dense(64, activation='relu')(regressor)
  gate_output = layers.Dense(8, name='Gate_Points')(dense)

  net = models.Model(gate_input, gate_output)
  return net

In [0]:
def residual_block(net_input, filter_size, filters):
  F1, F2, F3 = filters
  activation_shortcut = net_input

  Conv1 = layers.Conv2D(F1, 1)(net_input)
  Norm1 = layers.BatchNormalization(axis=-1)(Conv1)
  Act1 = layers.Activation('relu')(Norm1)

  Conv2 = layers.Conv2D(F2, kernel_size=filter_size, padding='same')(Act1)
  Norm2 = layers.BatchNormalization(axis=-1)(Conv2)
  Act2 = layers.Activation('relu')(Norm2)

  Conv3 = layers.Conv2D(F3, 1)(Act2)
  Norm3 = layers.BatchNormalization(axis=-1)(Conv3)
  Res = layers.Add()([Norm3, activation_shortcut])
  Act3 = layers.Activation('relu')(Res)

  return Act3

In [0]:
def conv_block(net_input, filter_size, filters, stride=2):
  F1, F2, F3 = filters
  activation_shortcut = net_input

  Conv1 = layers.Conv2D(F1, 1, strides=stride)(net_input)
  Norm1 = layers.BatchNormalization(axis=-1)(Conv1)
  Act1 = layers.Activation('relu')(Norm1)

  Conv2 = layers.Conv2D(F2, kernel_size=filter_size, padding='same')(Act1)
  Norm2 = layers.BatchNormalization(axis=-1)(Conv2)
  Act2 = layers.Activation('relu')(Norm2)

  Conv3 = layers.Conv2D(F3, 1)(Act2)
  Norm3 = layers.BatchNormalization(axis=-1)(Conv3)

  Short_cut = layers.Conv2D(F3, 1, strides=stride)(activation_shortcut)
  Short_cut = layers.BatchNormalization(axis=-1)(Short_cut)

  Res = layers.Add()([Norm3, Short_cut])
  Act3 = layers.Activation('relu')(Res)

  return Act3

In [0]:
def ResNet():
  gate_input = Input(shape=(216, 324, 3), name='Gates')

  X = layers.Conv2D(64, (7, 7), strides = 2, name = 'Conv1')(gate_input)
  X = layers.BatchNormalization(axis = -1)(X)
  X = layers.Activation('relu')(X)
  X = layers.MaxPooling2D((3, 3), strides=2)(X)

  X = conv_block(X, 3, [64, 64, 256], 1)
  X = residual_block(X, 3, [64, 64, 256])
  X = residual_block(X, 3, [64, 64, 256])

  X = conv_block(X, 3, [128, 128, 512], 2)
  X = residual_block(X, 3, [128, 128, 512])
  X = residual_block(X, 3, [128, 128, 512])
  X = residual_block(X, 3, [128, 128, 512])

  X = conv_block(X, 3, [128, 128, 512], 2)
  X = residual_block(X, 3, [128, 128, 512])
  X = residual_block(X, 3, [128, 128, 512])
  X = residual_block(X, 3, [128, 128, 512])

  X = conv_block(X, 3, [256, 256, 1024], 2)
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])

  X = conv_block(X, 3, [256, 256, 1024], 2)
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])

  X = conv_block(X, 3, [512, 512, 2048], 2)
  X = residual_block(X, 3, [512, 512, 2048])
  X = residual_block(X, 3, [512, 512, 2048])
  X = layers.AveragePooling2D(2, padding='same')(X)

  X = layers.Flatten()(X)
  gate_output = layers.Dense(8)(X)

  net = models.Model(gate_input, gate_output, name='ResNet')
  return net

In [0]:
def CS_ResNet():
  gate_input = Input(shape=(216, 324, 3), name='Gates')

  X = layers.Conv2D(64, (7, 7), strides = 2, name = 'Conv1')(gate_input)
  X = layers.BatchNormalization(axis = -1)(X)
  X = layers.Activation('relu')(X)
  X = layers.MaxPooling2D((3, 3), strides=2)(X)

  X = conv_block(X, 3, [64, 64, 256], 1)
  X = residual_block(X, 3, [64, 64, 256])
  X = residual_block(X, 3, [64, 64, 256])

  X = conv_block(X, 3, [128, 128, 512], 2)
  X = residual_block(X, 3, [128, 128, 512])
  X = residual_block(X, 3, [128, 128, 512])
  X = residual_block(X, 3, [128, 128, 512])

  X = conv_block(X, 3, [256, 256, 1024], 2)
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])
  X = residual_block(X, 3, [256, 256, 1024])

  X = conv_block(X, 3, [512, 512, 2048], 2)
  X = residual_block(X, 3, [512, 512, 2048])
  X = residual_block(X, 3, [512, 512, 2048])
  X = layers.AveragePooling2D(2, padding='same')(X)

  X = layers.Flatten()(X)
  X = layers.Dense(32, activation='relu')(X)
  gate_output = layers.Dense(1, activation='sigmoid')(X)

  net = models.Model(gate_input, gate_output, name='ResNetCS')
  return net

In [0]:
def train(epochs, batch_size, model_version, plotmodel=False):
  model_path = '/content/drive/My Drive/Colab Notebooks/GateNet' + str(model_version) + '.h5'
  model_graph_path = 'GateNet' + str(model_version) + '.png'

  if model_version == 1:
    nn = naive_model()
    nn.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  elif model_version == 2:
    nn = educated_model()
    nn.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  elif model_version == 3:
    nn = functional_model()
    nn.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  elif model_version == 4:
    nn = residual_model()
    nn.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  elif model_version == 5:
    nn = better_residual_model()
    nn.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  elif model_version == 6:
    with tf.device('/device:GPU:0'):
      nn = ResNet()
      nn.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  elif model_version == 7:
    nn = ResNet()
    nn.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  elif model_version == 8:
    nn = CS_ResNet()
    nn.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  if plotmodel:
    plot_model(nn, show_shapes=False, to_file=model_graph_path)

  callback_list = [EarlyStopping(monitor='loss', patience=10),
                 ModelCheckpoint(filepath=model_path,
                                 monitor='val_loss', save_best_only=True),
                 ReduceLROnPlateau(monitor='loss', min_delta=0.05, cooldown=5, factor=0.005, patience=1, verbose=1)]

  training_data = nn.fit(x_train.astype(np.float32), y_train.astype(np.float32), epochs=epochs, batch_size=batch_size, shuffle=True, epochs=epochs, validation_data=(x_val, y_val), callbacks=callback_list)
  performance = nn.evaluate(x_test, y_test, batch_size=batch_size)

  t_loss = training_data.history['loss']
  v_loss = training_data.history['val_loss']
  print('Test MAE:', performance[1])
  return t_loss, v_loss, performance

In [0]:
naive_model_path = '/content/drive/My Drive/Colab Notebooks/GateNet1.h5' # old reliable
educated_model_path ='/content/drive/My Drive/Colab Notebooks/GateNet2.h5' # rising star
functional_model_path ='/content/drive/My Drive/Colab Notebooks/GateNet3.h5' # it just works
residual_model_path = '/content/drive/My Drive/Colab Notebooks/GateNet4.h5' # just like ayo used to make
better_residual_model_path = '/content/drive/My Drive/Colab Notebooks/GateNet5.h5' # with these upgrades.. you never stood a chance
ResNet_model_path = '/content/drive/My Drive/Colab Notebooks/GateNet/content/drive/My Drive/Colab Notebooks/GateNet6.h5'

train(100, 32, 8)